In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wFMVoarN4fdkfv0kNmCj6264Y8frg-mlGx3AZzzv-4BKcePlB9hNdA
Mounted at /content/drive


In [3]:
a_pos_5289 = "/content/drive/My Drive/pattern_fasta/A.thaliana5289_pos/kmer2/A.thaliana5289_pos_kmer_dnc_2.npy"
a_neg_5289 = "/content/drive/My Drive/pattern_fasta/A.thaliana5289_neg/kmer2/A.thaliana5289_neg_kmer_dnc.npy"
a_pos_1000 = "/content/drive/My Drive/pattern_fasta/A.thaliana1000indep_pos/kmer_dnc/A.thaliana1000indep_pos_kmer_dnc.npy"
a_neg_1000 = "/content/drive/My Drive/pattern_fasta/A.thaliana1000indep_neg/kmer_dnc/A.thaliana1000indep_neg_kmer_dnc.npy"

a_x_train_pos = np.load(a_pos_5289) 
a_x_train_neg = np.load(a_neg_5289) 
a_x_test_pos = np.load(a_pos_1000) 
a_x_test_neg = np.load(a_neg_1000) 


b_pos_5289 = "/content/drive/My Drive/pattern_fasta/A.thaliana5289_pos/pcpsednc/A.thaliana5289_pos_pspsednc.npy"
b_neg_5289 = "/content/drive/My Drive/pattern_fasta/A.thaliana5289_neg/pcpsednc/A.thaliana5289_neg_pcpsednc.npy"
b_pos_1000 = "/content/drive/My Drive/pattern_fasta/A.thaliana1000indep_pos/pcpsednc/A.thaliana1000indep_pos_pcpsednc.npy"
b_neg_1000 = "/content/drive/My Drive/pattern_fasta/A.thaliana1000indep_neg/pcpsednc/A.thaliana1000indep_neg_pcpsednc.npy"

b_x_train_pos = np.load(b_pos_5289) 
b_x_train_neg = np.load(b_neg_5289) 
b_x_test_pos = np.load(b_pos_1000) 
b_x_test_neg = np.load(b_neg_1000) 

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000) 

In [4]:
print(a_x_test_neg.shape)
print(a_x_train_neg.shape)

print(a_x_test_pos.shape)
print(a_x_train_pos.shape)

print()
print(b_x_test_neg.shape)
print(b_x_train_neg.shape)

print(b_x_test_pos.shape)
print(b_x_train_pos.shape)

(1000, 16)
(5289, 16)
(1000, 16)
(5289, 16)

(1000, 18)
(5289, 18)
(1000, 18)
(5289, 18)


In [5]:
a_x_training = np.concatenate((a_x_train_pos, a_x_train_neg), axis = 0)
a_x_test = np.concatenate((a_x_test_pos, a_x_test_neg), axis = 0)

b_x_training = np.concatenate((b_x_train_pos, b_x_train_neg), axis = 0)
b_x_test = np.concatenate((b_x_test_pos, b_x_test_neg), axis = 0)

y_training = np.concatenate((y_train_pos,y_train_neg))
y_test = np.concatenate((y_test_pos,y_test_neg))

In [6]:
print(a_x_training.shape)
print(a_x_test.shape)

print(b_x_training.shape)
print(b_x_test.shape)

print()
print(y_test.shape)
print(y_training.shape)



(10578, 16)
(2000, 16)
(10578, 18)
(2000, 18)

(2000,)
(10578,)


In [7]:
x_training = np.concatenate((a_x_training, b_x_training), axis = 1)

x_test = np.concatenate((a_x_test, b_x_test), axis = 1)

In [8]:
print(x_training.shape)
print(x_test.shape)

(10578, 34)
(2000, 34)


In [9]:
seed = 40
np.random.seed(seed)

kf = KFold(n_splits=5, random_state=seed, shuffle=True)

In [10]:
test_avg_acc_list = []
j = 1
for c in range(50,502,50):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 50
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.6385
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.607
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.627
Train Accuracy :  1.0 	Validation Accuracy :  0.9966903073286052 	Test Accuracy  0.6115
Train Accuracy :  1.0 	Validation Accuracy :  0.9971631205673759 	Test Accuracy  0.613
Avg Test Accuracy: 0.6194
Epoch 2/10 for C = 100
Train Accuracy :  1.0 	Validation Accuracy :  0.997164461247637 	Test Accuracy  0.634
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.6365
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.652
Train Accuracy :  1.0 	Validation Accuracy :  0.9957446808510638 	Test Accuracy  0.636
Train Accuracy :  1.0 	Validation Accuracy :  0.9966903073286052 	Test Accuracy  0.6415
Avg Test Accuracy: 0.64
Epoch 3/10 for C = 150
Train Accura

In [34]:
test_avg_acc_list = []
j = 1
for c in range(100,150,10):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 100
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.64
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.619
Train Accuracy :  1.0 	Validation Accuracy :  0.998109640831758 	Test Accuracy  0.6205
Train Accuracy :  1.0 	Validation Accuracy :  0.9957446808510638 	Test Accuracy  0.6225
Train Accuracy :  1.0 	Validation Accuracy :  0.9966903073286052 	Test Accuracy  0.6285
Avg Test Accuracy: 0.6260999999999999
Epoch 2/10 for C = 110
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.6395
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.6255
Train Accuracy :  1.0 	Validation Accuracy :  0.997164461247637 	Test Accuracy  0.6385
Train Accuracy :  1.0 	Validation Accuracy :  0.9957446808510638 	Test Accuracy  0.6185
Train Accuracy :  1.0 	Validation Accuracy :  0.9962174940898345 	Test Accuracy  0.641
Avg Test Accuracy: 0.6325999999999999
Epoch

In [11]:
test_avg_acc_list = []
j = 1
for c in range(5,106,10):
  RFC = RandomForestClassifier(n_estimators=270,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{15} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/15 for C = 5
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.6305
Train Accuracy :  1.0 	Validation Accuracy :  0.9985822306238186 	Test Accuracy  0.6195
Train Accuracy :  1.0 	Validation Accuracy :  0.997164461247637 	Test Accuracy  0.629
Train Accuracy :  1.0 	Validation Accuracy :  0.9962174940898345 	Test Accuracy  0.6255
Train Accuracy :  1.0 	Validation Accuracy :  0.9966903073286052 	Test Accuracy  0.6435
Avg Test Accuracy: 0.6296
Epoch 2/15 for C = 15
Train Accuracy :  1.0 	Validation Accuracy :  0.9976370510396976 	Test Accuracy  0.6515
Train Accuracy :  1.0 	Validation Accuracy :  0.999054820415879 	Test Accuracy  0.626
Train Accuracy :  1.0 	Validation Accuracy :  0.997164461247637 	Test Accuracy  0.6265
Train Accuracy :  1.0 	Validation Accuracy :  0.9966903073286052 	Test Accuracy  0.6255
Train Accuracy :  1.0 	Validation Accuracy :  0.9976359338061466 	Test Accuracy  0.643
Avg Test Accuracy: 0.6344999999999998
Epoch 3/15 for C = 